In [1]:
import os
import torch

from dotenv import load_dotenv

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

d:\Visual Code\RAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Variables

In [2]:
load_dotenv()

True

In [3]:
file = os.getenv("FILE")
persist_directory = os.getenv("PERSIST_DIRECTORY") + "/faiss"
token = os.getenv("TOKEN")
model_id = os.getenv("MODEL_ID")
model_name = os.getenv("MODEL_NAME")

# PDFLoader

In [4]:
loader = PyPDFLoader(file)
file = loader.load()

# Recursive TextSplit

In [5]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150
)

In [6]:
documents = recur_split.split_documents(file)

# Embeddings (HuggingFace)

In [7]:
embedding = HuggingFaceEmbeddings(
    model_name=model_name, 
    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}, 
    encode_kwargs={'normalize_embeddings': False}
)

# Vector Store

In [8]:
vectordb = FAISS.from_documents(
    documents=documents, 
    embedding=embedding
)

vectordb.save_local(persist_directory)

# LLM (HuggingFace)

In [9]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=token,
    quantization_config=quantization_config    
)

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    token=token    
)

pipe = pipeline(
    model=model, 
    tokenizer=tokenizer, 
    task="text-generation",
    return_full_text=False
)

llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={
        "temperature": 0.0,        
        "do_sample": False        
    }
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.50s/it]
Device set to use cuda:0


# Prompt

In [10]:
token_start, token_end = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>", "<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

template = """
Você é um assistente educacional que retorna competências empreendedoras.

O usuário fornecerá os dados de uma aula em formato estruturado, incluindo:
**Disciplina**;
**Conteúdo**;
**Objetivo da Aula**.

Com base no documento (que está no contexto abaixo), identifique **três competências empreendedoras** que se relacionam diretamente com essa aula.
Retorne apenas os nomes das competências, separados por vírgula sendo a última separada por 'e'. \n\n

Exemplo: \n
Pergunta: "**Disciplina** <Matemática>, sendo o **Conteúdo** <Frações> e o **Objetivo da Aula** é <Calcular juros>." \n
Resposta: Identificar oportunidades, Mobilização de recursos e Educação financeira e econômica. \n\n

Contexto: {context} \n\n
Pergunta: {input}
"""

prompt = ChatPromptTemplate.from_template(token_start + template + token_end)

# Chain

In [11]:
documents_chain = create_stuff_documents_chain(
    llm, 
    prompt
)

retriver_chain = create_retrieval_chain(
    retriever=vectordb.as_retriever(search_type="mmr"), 
    combine_docs_chain=documents_chain
)

# Question

In [12]:
question = "**Disciplina** <Computação>, sendo o **Conteúdo** <Criação de Programas> e o **Objetivo da Aula** é <Desenvolvimento de sites Web>."

# Answer

In [13]:
retriver_chain.invoke({"input": question})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'input': '**Disciplina** <Computação>, sendo o **Conteúdo** <Criação de Programas> e o **Objetivo da Aula** é <Desenvolvimento de sites Web>.',
 'context': [Document(id='d20ee17c-38c3-4ae7-844b-eaac06be9867', metadata={'producer': 'Microsoft® Word para Microsoft 365', 'creator': 'Microsoft® Word para Microsoft 365', 'creationdate': '2025-04-28T16:59:30-03:00', 'author': 'Roselaine Monteiro Moraes', 'moddate': '2025-04-28T16:59:30-03:00', 'source': '../pdf/entrecomp.pdf', 'total_pages': 2, 'page': 1, 'page_label': '2'}, page_content='organizar e acompanhar. Definir objetivos de longo, médio e curto prazo, definindo \nprioridades e planos de ação, adaptando-se quando necessário à mudanças imprevistas. \n \nLidar com a incerteza: a ambiguidade e o risco é tomar decisões mesmo quando \nos seus resultados são incertos, quando a informação disponível é parcial ou ambígua, \nou quando existe um risco de resultados indesejados. No processo de criação de valor, \nincluir formas estruturadas de